In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida

import ipywidgets as ipw
import traitlets
import warnings

from IPython.display import clear_output

from aiidalab_widgets_base import OptimadeQueryWidget
from aiidalab_widgets_base import StructureBrowserWidget
from aiidalab_widgets_base import StructureManagerWidget
from aiidalab_widgets_base import StructureUploadWidget
from aiidalab_widgets_base import SmilesWidget
from aiidalab_widgets_base import ComputationalResources
from aiidalab_widgets_base import SubmitButtonWidget

import aiida_nanotech_empa.utils.gaussian_wcs_postprocess as pp
StructureData = DataFactory("structure")
GaussianSpinWorkChain = WorkflowFactory('nanotech_empa.gaussian.spin')

from aiidalab_widgets_base import WizardAppWidget
from aiidalab_widgets_base import WizardAppWidgetStep

In [ ]:
structure_manager_widget = StructureManagerWidget(
    importers=[
        StructureUploadWidget(title="From computer"),
        OptimadeQueryWidget(embedded=True),
        StructureBrowserWidget(title="AiiDA database"),
        SmilesWidget(title="SMILES")
    ],
    node_class='StructureData',
)

class StructureSelectionStep(ipw.VBox, WizardAppWidgetStep):
    """Integrated widget for the selection of structures from different sources."""

    structure = traitlets.Instance(StructureData, allow_none=True)
    confirmed_structure = traitlets.Instance(StructureData, allow_none=True)

    def __init__(self, manager, description=None, **kwargs):
        self.manager = manager

        if description is None:
            description = ipw.HTML(
                """
                <p>Select a structure from one of the following sources and then click
                "Confirm" to go to the next step.
                """
            )
        self.description = description

        self.structure_name_text = ipw.Text(
            placeholder="[No structure selected]",
            description="Selected:",
            disabled=True,
            layout=ipw.Layout(width="auto", flex="1 1 auto"),
        )

        self.confirm_button = ipw.Button(
            description="Confirm",
            tooltip="Confirm the currently selected structure and go to the next step.",
            button_style="success",
            icon="check-circle",
            disabled=True,
            layout=ipw.Layout(width="auto"),
        )
        self.confirm_button.on_click(self.confirm)
        self.message_area = ipw.HTML()

        # Create directional link from the (read-only) 'structure_node' traitlet of the
        # structure manager to our 'structure' traitlet:
        ipw.dlink((manager, "structure_node"), (self, "structure"))

        super().__init__(
            children=[
                self.description,
                self.manager,
                self.structure_name_text,
                self.message_area,
                self.confirm_button,
            ],
            **kwargs
        )

    @traitlets.default("state")
    def _default_state(self):
        return self.State.INIT

    def _update_state(self):
        if self.structure is None:
            if self.confirmed_structure is None:
                self.state = self.State.READY
            else:
                self.state = self.State.SUCCESS
        else:
            if self.structure.pbc != (True, True, True):
                self.state = self.State.READY
            elif self.confirmed_structure is None:
                self.state = self.State.CONFIGURED
            else:
                self.state = self.State.SUCCESS

    @traitlets.observe("structure")
    def _observe_structure(self, change):
        structure = change["new"]
        with self.hold_trait_notifications():
            if structure is None:
                self.structure_name_text.value = ""
                self.message_area.value = ""
            else:
                self.structure_name_text.value = str(self.structure.get_formula())
            self._update_state()

    @traitlets.observe("confirmed_structure")
    def _observe_confirmed_structure(self, _):
        with self.hold_trait_notifications():
            self._update_state()

    @traitlets.observe("state")
    def _observe_state(self, change):
        with self.hold_trait_notifications():
            state = change["new"]
            self.confirm_button.disabled = state != self.State.CONFIGURED
            self.manager.disabled = state is self.State.SUCCESS

    def confirm(self, _=None):
        self.manager.store_structure()
        self.confirmed_structure = self.structure
        self.message_area.value = ""

    def can_reset(self):
        return self.confirmed_structure is not None

    def reset(self):  # unconfirm
        self.confirmed_structure = None
        
select_structure_step = StructureSelectionStep(manager=structure_manager_widget, auto_advance=True)


In [ ]:
class ConfigureGaussianCalculationStep(ipw.VBox, WizardAppWidgetStep):
    """Integrated widget for the selection of structures from different sources."""


    # We use traitlets to connect the different steps.
    # Note that we can use dlinked transformations, they do not need to be of the same type.
    inputs = traitlets.Dict()
    input_structure = traitlets.Instance(StructureData, allow_none=True)

    def __init__(self, **kwargs):

        self.dft_functional = ipw.Dropdown(options=['PBE', 'PBE0'])

        self.observe(self._update_state, ["inputs", "input_structure"])
        
        self.confirm_button = ipw.Button(
            description="Confirm",
            tooltip="Confirm the currently selected structure and go to the next step.",
            button_style="success",
            icon="check-circle",
            disabled=False,
            layout=ipw.Layout(width="auto"),
        )
        self.confirm_button.on_click(self.confirm)

        super().__init__([self.dft_functional, self.confirm_button], **kwargs)
        
    def reset(self):
        self.inputs = {}

    def submit_order(self, button):
        "Submit the order and simulate the delivery."
        self.configuration = {}

    def _update_state(self, _=None):
        "Update the step's state based on the order status and configuration traits."
        if self.input_structure:  # the order can be submitted
            self.state = self.State.READY
        else:
            self.state = self.State.INIT

    def confirm(self, _=None):
        self.state = self.State.SUCCESS
    
    @traitlets.default("state")
    def _default_state(self):
        return self.State.INIT

configure_calculation_step = ConfigureGaussianCalculationStep(auto_advance=True)

ipw.dlink((select_structure_step, 'confirmed_structure'), (configure_calculation_step, 'input_structure'))


In [ ]:
class SubmitGaussianCalculationStep(ipw.VBox, WizardAppWidgetStep):
    """Integrated widget for the selection of structures from different sources."""


    # We use traitlets to connect the different steps.
    # Note that we can use dlinked transformations, they do not need to be of the same type.
    configuration = traitlets.Dict()

    def __init__(self, **kwargs):
        # The pizza configuration is represented as a formatted dictionary.
        self.configuration_label = ipw.HTML()
        
        self.gaussian_code_dropdown = ComputationalResources(input_plugin='gaussian')
        self.cubegen_code_dropdown = ComputationalResources(input_plugin='gaussian.cubegen')
        self.formchk_code_dropdown = ComputationalResources(input_plugin='gaussian.formchk')

        # We update the step's state whenever there is a change to the configuration or the order status.
        self.observe(self._update_state, ["configuration"])
        
        btn_submit_mol_opt = SubmitButtonWidget(GaussianSpinWorkChain, input_dictionary_function=self.prepare_spin_calc)

        super().__init__([self.gaussian_code_dropdown,
                          self.cubegen_code_dropdown,
                          self.formchk_code_dropdown, btn_submit_mol_opt], **kwargs)
        
    def reset(self):
        self.configuration ={}

    @traitlets.observe("configuration")
    def _observe_configuration(self, change):
        "Format and show the pizza configuration."
        if change["new"]:
            self.configuration_label.value = f"<h4>Configuration</h4><pre>{json.dumps(change['new'], indent=2)}</pre>"
        else:
            self.configuration_label.value = (
                "<h4>Configuration</h4>[Please configure your pizza]"
            )

    def _update_state(self, _=None):
        "Update the step's state based on the order status and configuration traits."
        if self.configuration:  # the order can be submitted
            self.state = self.State.CONFIGURED
        else:
            self.state = self.State.INIT
    
    def prepare_spin_calc(self):
        builder = GaussianSpinWorkChain.get_builder()
        builder.gaussian_code = self.gaussian_code_dropdown.selected_code
        builder.formchk_code = self.formchk_code_dropdown.selected_code
        builder.cubegen_code = self.cubegen_code_dropdown.selected_code
        builder.structure = structure_manager_widget.structure_node
        builder.functional = Str('B3LYP')
        builder.empirical_dispersion = Str('GD3')
        builder.basis_set_opt = Str('STO-3G')
        builder.basis_set_scf = Str('STO-3G')
        builder.multiplicity_list = List(list=[0, 1, 3])
        
        
        self.state = self.State.SUCCESS

        return builder

submit_calculation_step = SubmitGaussianCalculationStep(auto_advance=True)

In [ ]:
class ResultsStep(ipw.VBox, WizardAppWidgetStep):


    # We use traitlets to connect the different steps.
    # Note that we can use dlinked transformations, they do not need to be of the same type.

    def __init__(self, **kwargs):
        # The pizza configuration is represented as a formatted dictionary.
        self.calculations = ipw.Dropdown(description="Select calc PK:", options=[966, 1284])
        self.calculations.observe(self.show_results, ["value"])

        self._output = ipw.Output()

        super().__init__([self.calculations,
                          self._output], **kwargs)
        
    def reset(self):
        self.workchain = None

    def show_results(self, _=None):
        with self._output:
            clear_output()
            wc_node = load_node(self.calculations.value)
            pp.make_report(wc_node, nb=True)
            

results = ResultsStep(auto_advance=True)

In [ ]:
app = WizardAppWidget(
    steps=[
        ("Select structure", select_structure_step),
        ("Cofigure calculation", configure_calculation_step),
        ("Submit", submit_calculation_step),
        ("Results", results)
    ]
)

In [ ]:
display(app)